# Imports

In [1]:
import numpy as np
import pandas as pd
import numexpr as ne

In [181]:
from simplenn.activations import LRelu
from simplenn.activations import Sigmoid
from simplenn.loss import Square
from simplenn.loss import NegLogLike
from simplenn.network import Network
from simplenn.network import Layer
from simplenn.optim import BackProp
from simplenn.optim import Genetic

In [3]:
%matplotlib

Using matplotlib backend: TkAgg


# Data

In [4]:
xs = np.array(
    [
        [0,0,0,0,0,0,0,0,0,1],
        [0,0,0,0,0,0,0,0,1,0],
        [0,0,0,0,0,0,0,1,0,0],
        [0,0,0,0,0,0,1,0,0,0],
        [0,0,0,0,0,1,0,0,0,0],
        [0,0,0,0,1,0,0,0,0,0],
        [0,0,0,1,0,0,0,0,0,0],
        [0,0,1,0,0,0,0,0,0,0],
        [0,1,0,0,0,0,0,0,0,0],
        [1,0,0,0,0,0,0,0,0,0]
    ],
    dtype='float64'
).T

ys = np.array(
    [
        [0,0,0,0],
        [0,0,0,1],
        [0,0,1,0],
        [0,0,1,1],
        [0,1,0,0],
        [0,1,0,1],
        [0,1,1,0],
        [0,1,1,1],
        [1,0,0,0],
        [1,0,0,1]
    ],
    dtype='float64'
).T

# Experiments

### Genetic Training

In [277]:
popSize = 50
nElitism = 0
generations = 500
tournamentSize = 5
mutationProba = 0.5
mutationScale = 0.05
mutationRelative = False
verboseFreq = 50
recordFreq = 50

In [281]:
net = Network(
    NegLogLike(),
    [
        Layer(xs.shape[0], 7, Sigmoid(), "kaiming"),
        Layer(7, 7, Sigmoid(), "kaiming"),
        Layer(7, ys.shape[0], Sigmoid(), "kaiming", True)
    ]
)

In [282]:
genetic = Genetic()
net = genetic.run(
    net, 
    xs, 
    ys, 
    popSize, 
    tournamentSize,
    nElitism, 
    mutationProba, 
    mutationScale, 
    mutationRelative, 
    generations, 
    verboseFreq, 
    recordFreq)

Generation 0: 25.91322245088901
Generation 50: 20.890265081262186
Generation 100: 14.755608441781513
Generation 150: 9.20158141349874
Generation 200: 4.730064556507463
Generation 250: 2.1887633224844505
Generation 300: 0.9833695592494265
Generation 350: 0.44357320368785425
Generation 400: 0.20361557358925475
Generation 450: 0.09256175199798974


In [283]:
net.forward(xs[:,:9]).round(2)

array([[0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 1., 1., 1., 0.],
       [0., 0., 1., 1., 0., 0., 1., 1., 0.],
       [0., 1., 0., 1., 0., 1., 0., 1., 0.]])

### Gradient Back-Propagation Training

In [216]:
lRate = 0.1
batchSize = xs.shape[1]
epochs = 100*1000
verboseFreq = 10*1000
recordFreq = 10*1000

In [217]:
net = Network(
    NegLogLike(),
    [
        Layer(xs.shape[0], 7, Sigmoid(), "xavier"),
        Layer(7, 7, Sigmoid(), "xavier"),
        Layer(7, 7, Sigmoid(), "xavier"),
        Layer(7, 7, Sigmoid(), "xavier"),
        Layer(7, ys.shape[0], Sigmoid(), "xavier", True)
    ]
)

In [218]:
bp = BackProp()
bp.run(net, xs, ys, batchSize, lRate, epochs, verboseFreq, recordFreq)

Epoch 0: 29.25946514444577
Epoch 10000: 16.779396662037254
Epoch 20000: 1.254592869198135
Epoch 30000: 0.17017421809823
Epoch 40000: 0.0911041516415434
Epoch 50000: 0.06221930847019495
Epoch 60000: 0.04721026618189346
Epoch 70000: 0.03801108867921158
Epoch 80000: 0.03179621825331623
Epoch 90000: 0.0273172823715613


In [219]:
net.forward(xs[:,:9]).round(2)

array([[0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 1., 1., 1., 0.],
       [0., 0., 1., 1., 0., 0., 1., 1., 0.],
       [0., 1., 0., 1., 0., 1., 0., 1., 0.]])

### Network exploration

In [232]:
df = pd.DataFrame({
    f"Layer-{i}":pd.Series([np.abs(n.layers[i].W).sum() for n in genetic.networks])
    for i in range(len(genetic.networks[0].layers))
}).plot(title='Weights Norm Evolution')

In [221]:
df = pd.DataFrame({
    f"Layer-{i}":pd.Series([np.abs(n.layers[i].delta_W).sum() for n in bp.networks])
    for i in range(len(bp.networks[0].layers))
}).plot(title='Gradient Norm Evolution')

In [206]:
net = bp.networks[10]
df = pd.concat(
    [pd.Series(net.layers[i].W.flatten(), name=f"W{i}") for i in range(len(net.layers))],
    axis=1
)
df.hist(bins=20)

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7fe3ca7d9f50>,
      dtype=object)

In [207]:
net = bp.networks[10]
df = pd.concat(
    [pd.Series(net.layers[i].A.flatten(), name=f"A{i}") for i in range(len(net.layers))],
    axis=1
)
df.hist(bins=20)

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7fe3ca5962d0>,
      dtype=object)

In [208]:
net = bp.networks[10]
df = pd.concat(
    [pd.Series(net.layers[i].delta_W.flatten(), name=f"delta_W{i}") for i in range(len(net.layers))],
    axis=1
)
df.hist(bins=20)

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7fe3ca7e7e50>,
      dtype=object)

### Numexpr vs. Numpy

In [10]:
%%timeit
A = np.random.random((10000, 5000))
for _ in range(10):
    A = ne.evaluate("cos(A)")

2.46 s ± 183 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
A = np.random.random((10000, 5000))
for _ in range(10):
    A = np.cos(A)

6.07 s ± 155 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
